In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
dirty_data = pd.read_csv('/content/opinie.csv')

In [ ]:
def rescale_rank(rank: int):
  if rank > 3:
    return 1
  elif rank < 3:
    return 0
  else:
    return 2

In [ ]:
dirty_data['ocena'] = dirty_data['ocena'].apply(lambda x: rescale_rank(int(x[7])))

In [ ]:
dirty_data.head()

,strona,opinia,ocena
0,https://www.motofakty.pl/samochody/opinie/aeon...,Lpl,0
1,https://www.motofakty.pl/samochody/opinie/acur...,MA SWOJE LATA (2006) ALE OK.,1
2,https://www.motofakty.pl/samochody/opinie/acur...,NO KUPIŁBYM ZADBANEGO,1
3,https://www.motofakty.pl/samochody/opinie/aust...,Polecam super klasyk,1
4,https://www.motofakty.pl/samochody/opinie/abar...,"samochód ładny,bezpieczny,szybki.",1


In [ ]:
dirty_data.ocena.value_counts()

1    35975
2      899
0      803
Name: ocena, dtype: int64

In [ ]:
non_positive = dirty_data.loc[dirty_data.ocena != 1]

In [ ]:
positive = dirty_data.loc[dirty_data.ocena == 1].sample(1500, random_state=42)

In [ ]:
moto_data = positive.append(non_positive)

In [ ]:
with open('/content/stopwords.txt', "r") as f:
  STOP_WORDS = set([line.rstrip("\n") for line in f])

In [ ]:
def preprocess(intext: str):
  text = re.sub(r'\W+', ' ', intext.lower()).split()
  text = " ".join([word for word in text if word not in STOP_WORDS])

  return text


In [ ]:
moto_data['opinia'] = moto_data['opinia'].apply(lambda x: preprocess(x))

In [ ]:
moto_data.head()

,strona,opinia,ocena
32387,https://www.motofakty.pl/samochody/opinie/toyo...,auto zadba ipodokreca morzna cieszyc przyjemno...,1
3163,https://www.motofakty.pl/samochody/opinie/bmw/...,bmw b 12 750 spalanie miasto 19l 100 100 km h ...,1
16910,https://www.motofakty.pl/samochody/opinie/mazd...,wdzieczny pojazd,1
14056,https://www.motofakty.pl/samochody/opinie/isuz...,całkiem dobry samochód mogę polecić jedynie po...,1
19701,https://www.motofakty.pl/samochody/opinie/niss...,kupił bym ponownie auto,1


In [ ]:
moto_data.to_csv('/content/drive/My Drive/moto_data_preprocessed.csv')

In [ ]:
!pip install transformers==2.8.0 -q

     |████████████████████████████████| 573kB 13.3MB/s 
     |████████████████████████████████| 1.1MB 55.7MB/s 
     |████████████████████████████████| 3.7MB 54.1MB/s 
     |████████████████████████████████| 890kB 47.0MB/s 
     |████████████████████████████████| 133kB 56.6MB/s 
     |████████████████████████████████| 6.9MB 52.8MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
ERROR: botocore 1.19.25 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
from transformers import *
import torch

In [ ]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

#model.cuda()
device = torch.device("cuda")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
import pandas as pd

moto_data = pd.read_csv('/content/drive/My Drive/moto_data_preprocessed.csv', index_col=0)

In [ ]:
moto_data.head()

,strona,opinia,ocena
32387,https://www.motofakty.pl/samochody/opinie/toyo...,auto zadba ipodokreca morzna cieszyc przyjemno...,1
3163,https://www.motofakty.pl/samochody/opinie/bmw/...,bmw b 12 750 spalanie miasto 19l 100 100 km h ...,1
16910,https://www.motofakty.pl/samochody/opinie/mazd...,wdzieczny pojazd,1
14056,https://www.motofakty.pl/samochody/opinie/isuz...,całkiem dobry samochód mogę polecić jedynie po...,1
19701,https://www.motofakty.pl/samochody/opinie/niss...,kupił bym ponownie auto,1


In [ ]:
import numpy as np

np.max(moto_data.opinia.str.split().map(len))

607

In [ ]:
moto_data.ocena.value_counts()

1    1500
2     899
0     803
Name: ocena, dtype: int64

In [ ]:
moto_data.opinia = moto_data.opinia.astype(str)
moto_data.ocena = moto_data.ocena.astype(int)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

X = moto_data.opinia.to_numpy()
y = moto_data.ocena.to_numpy()
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in sss.split(X, y):
  train_sentences, test_sentences = X[train_index], X[test_index]
  train_labels, test_labels = y[train_index], y[test_index]

X = test_sentences
y = test_labels
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.33, random_state=42)
for train_index, test_index in sss.split(X, y):
  dev_sentences, test_sentences = X[train_index], X[test_index]
  dev_labels, test_labels = y[train_index], y[test_index]

In [ ]:
print('Train data')
print(len(train_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))
print('Dev data')
print(len(dev_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))
print('Test data')
print(len(test_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))

Train data
0.6998750780762024
Dev data
0.20081199250468457
Test data
0.09931292941911306


In [ ]:
dev_moto_data = pd.DataFrame([dev_sentences,dev_labels]).T
test_moto_data = pd.DataFrame([test_sentences,test_labels]).T

In [ ]:
dev_moto_data.to_csv('/content/drive/My Drive/dev_moto_data_preprocessed.csv')
test_moto_data.to_csv('/content/drive/My Drive/test_moto_data_preprocessed.csv')

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

X = train_sentences
y = train_labels
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X, y):
  train_sentences, test_sentences = X[train_index], X[test_index]
  train_labels, test_labels = y[train_index], y[test_index]

X = test_sentences
y = test_labels
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for train_index, test_index in sss.split(X, y):
  dev_sentences, test_sentences = X[train_index], X[test_index]
  dev_labels, test_labels = y[train_index], y[test_index]

In [ ]:
print('Train data')
print(len(train_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))
print('Dev data')
print(len(dev_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))
print('Test data')
print(len(test_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))

Train data
0.7996430165104864
Dev data
0.0999553770638108
Test data
0.10040160642570281


In [ ]:
# Remove long sentences.
# TO-DO Possible cut?
def remove_big(sentences, labels):
  to_remove = []
  for i, sent in enumerate(sentences):
      input_ids = tokenizer.encode(sent, add_special_tokens=True) # TO-DO: add_special_tokens
      if len(input_ids) > MAX_LEN:
        to_remove.append(i)

  sentences = np.delete(sentences, to_remove)
  labels = np.delete(labels, to_remove) 

  print('{} samples removed.'.format(len(to_remove)))

  return sentences, labels

In [ ]:
# Downloading tokenizer
# From Polbert - Polish BERT by Darek Kłeczek: https://github.com/kldarek/polbert
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

In [ ]:
train_sentences.dtype

dtype('O')

In [ ]:
MAX_LEN = 128

train_sentences, train_labels = remove_big(train_sentences, train_labels)
test_sentences, test_labels = remove_big(test_sentences, test_labels)
dev_sentences, dev_labels = remove_big(dev_sentences, dev_labels)

51 samples removed.
9 samples removed.
7 samples removed.


In [ ]:
from torch.utils.data import TensorDataset
# Create TensorDatasets for train/dev/test sets
def tensor_dataset(sentences, labels):
  input_ids = []
  attention_masks = []

  for sent in sentences:
      encoded_dict = tokenizer.encode_plus(
                          sent,                     
                          add_special_tokens = True,
                          max_length = MAX_LEN,
                          pad_to_max_length = True,
                          return_attention_mask = True,
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      input_ids.append(encoded_dict['input_ids'])
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)
  dataset = TensorDataset(input_ids, attention_masks, labels)
  return dataset

In [ ]:
BATCH_SIZE = 8

train_dataset = tensor_dataset(train_sentences, train_labels)
test_dataset = tensor_dataset(test_sentences, test_labels)
dev_dataset = tensor_dataset(dev_sentences, dev_labels)

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# Create the DataLoaders for train/dev/test sets.
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = BATCH_SIZE)
validation_dataloader = DataLoader(dev_dataset, sampler = SequentialSampler(dev_dataset), batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = BATCH_SIZE)

In [ ]:
# Load model with a sequence classification head
model = BertForSequenceClassification.from_pretrained(
    "dkleczek/bert-base-polish-uncased-v1", # Polbert - Polish BERT by Darek Kłeczek: https://github.com/kldarek/polbert
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(60000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
import time, datetime
import numpy as np
from tqdm import tqdm
from transformers.optimization import AdamW
from transformers import AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Takes a time in seconds and returns a string hh:mm:ss
def format_time(elapsed):
  elapsed_rounded = int(round((elapsed)))
  return str(datetime.timedelta(seconds=elapsed_rounded))

# Parameters:
epochs = 3
#lr = 1e-3 # Learning rate (Adam): 5e-5, 3e-5, 2e-5
lr = 5e-5 # Learning rate (Adam): 5e-5, 3e-5, 2e-5
adam_epsilon = 1e-10
WARM_UP = 0

optimizer = AdamW(model.parameters(), lr = lr, eps = adam_epsilon)

from transformers import get_linear_schedule_with_warmup
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = WARM_UP, num_training_steps = total_steps)

train_loss_values = []
dev_acc_values = []

model.zero_grad()

t0 = time.time()
for epoch_i in range(0, epochs):  
  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')

  # https://github.com/huggingface/transformers/blob/master/examples/run_glue.py
  # linie 168-183
  epoch_train_loss = 0 # Cumulative loss
  loss = 0 ;     batch_loss = 0
  model.train()

  for step, batch in enumerate(train_dataloader):

    # Progress update every 40 batches.
    if step % 40 == 0 and not step == 0:
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)      
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}. Loss: {:.3f}  Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))
    

    batch_loss = 0
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)    

    # clear any previously calculated gradients before backward pass
    optimizer.zero_grad()

    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

    loss = outputs[0]
    epoch_train_loss += loss.item()
    batch_loss += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
    optimizer.step()
    scheduler.step()  # Update learning rate schedule

  epoch_train_loss = epoch_train_loss / len(train_dataloader)          
  train_loss_values.append(epoch_train_loss)
  
  print('Average training loss: {0:.2f}'.format(epoch_train_loss))

  # Evaluation
  total_eval_accuracy = 0
  model.eval()

  for batch in validation_dataloader:
    
    input_ids = batch[0].to(device)
    attention_masks = batch[1].to(device)
    labels = batch[2].to('cpu').numpy()
                
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()

    predictions = np.argmax(logits, axis=1).flatten()
    total_eval_accuracy += flat_accuracy(logits, labels)

  avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
  print("  Accuracy: {0:.4f}".format(avg_val_accuracy))



======== Epoch 1 / 3 ========
Training...


/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


  Batch    40  of    218. Loss: 0.808  Elapsed: 0:00:08.
  Batch    80  of    218. Loss: 1.689  Elapsed: 0:00:17.
  Batch   120  of    218. Loss: 0.841  Elapsed: 0:00:25.
  Batch   160  of    218. Loss: 1.253  Elapsed: 0:00:34.
  Batch   200  of    218. Loss: 0.574  Elapsed: 0:00:43.
Average training loss: 0.95
  Accuracy: 0.6518

======== Epoch 2 / 3 ========
Training...
  Batch    40  of    218. Loss: 0.243  Elapsed: 0:00:57.
  Batch    80  of    218. Loss: 0.387  Elapsed: 0:01:05.
  Batch   120  of    218. Loss: 1.098  Elapsed: 0:01:14.
  Batch   160  of    218. Loss: 0.506  Elapsed: 0:01:23.
  Batch   200  of    218. Loss: 1.042  Elapsed: 0:01:32.
Average training loss: 0.63
  Accuracy: 0.5982

======== Epoch 3 / 3 ========
Training...
  Batch    40  of    218. Loss: 0.759  Elapsed: 0:01:46.
  Batch    80  of    218. Loss: 0.196  Elapsed: 0:01:55.
  Batch   120  of    218. Loss: 0.113  Elapsed: 0:02:04.
  Batch   160  of    218. Loss: 0.321  Elapsed: 0:02:13.
  Batch   200  of    2

In [ ]:
predicted_labels = [] ; true_labels = []; logits_list = []

for batch in test_dataloader:
  
  input_ids = batch[0].to(device)
  attention_masks = batch[1].to(device)
  labels = batch[2]
  
  with torch.no_grad():        
      outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
                  
  logits = outputs[0]
  logits = logits.detach().cpu().numpy()
  logits_list.append(logits)
  
  predictions = np.argmax(logits, axis=1).flatten()
  labels = labels.numpy().flatten()

  predicted_labels.extend( predictions )
  true_labels.extend( labels )
  

In [ ]:
# Parameters:
#BATCH = 16
#epochs = 3
#lr = 2e-5 # Learning rate (Adam): 5e-5, 3e-5, 2e-5
#adam_epsilon = 1e-8
#WARM_UP = 0.2
# acc 0.72 f1 0.72
from sklearn.metrics import classification_report 
print( classification_report(y_true=true_labels, y_pred=predicted_labels, zero_division=0) )

              precision    recall  f1-score   support

           0       0.62      0.47      0.54        55
           1       0.77      0.76      0.76       100
           2       0.45      0.56      0.50        61

    accuracy                           0.63       216
   macro avg       0.61      0.60      0.60       216
weighted avg       0.64      0.63      0.63       216



In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from transformers import WEIGHTS_NAME, CONFIG_NAME
output_dir = "/content/drive/My Drive/model_bert_finetuned_2"

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned
# If we have a distributed model, save only the encapsulated model
# (it was wrapped in PyTorch DistributedDataParallel or DataParallel)

model_to_save = model.module if hasattr(model, 'module') else model
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_pretrained(output_dir)

('/content/drive/My Drive/model_bert_finetuned_2/vocab.txt',
 '/content/drive/My Drive/model_bert_finetuned_2/special_tokens_map.json',
 '/content/drive/My Drive/model_bert_finetuned_2/added_tokens.json')

In [ ]:
# Step 2: Re-load the saved model and vocabulary
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)